In [34]:
from glob import glob
import json
import pandas as pd
from cvss import CVSS3
import datetime

In [47]:
PATH = "../../advisory-database/advisories/github-reviewed"

df = pd.DataFrame(columns=["date", "package", "ecosystem", "score"])
for i, file in enumerate(glob(f"{PATH}/**/*.json", recursive=True)):
    with open(file, "r") as f:
        data = json.load(f)

    date = datetime.datetime.fromisoformat(data["published"])
    if len(data["severity"]) > 0:
        vector = data["severity"][0]["score"]
        score = CVSS3(vector).base_score
    else:
        name = data["database_specific"]["severity"]
        match name:
            case "LOW":
                score = 3
            case "MODERATE":
                score = 5
            case "HIGH":
                score = 8
            case "CRITICAL":
                score = 10

    for package in data["affected"]:
        package = package["package"]
        eco = package.get("ecosystem", None)
        if eco in ["crates.io", "npm", "PyPI"]:
            df = pd.concat(
                (
                    df,
                    pd.DataFrame(
                        [[date, package["name"], eco, score]], columns=df.columns
                    ),
                )
            )

/tmp/ipykernel_28740/733834047.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


In [49]:
npm_df = df[df["ecosystem"] == "npm"]
cargo_df = df[df["ecosystem"] == "crates.io"]
pypi_df = df[df["ecosystem"] == "PyPI"]

In [53]:
npm_df.to_csv("npm.csv", index=False)
cargo_df.to_csv("cargo.csv", index=False)
pypi_df.to_csv("pypi.csv", index=False)